# Hello PyBullet World

Para entender el simulador creado por Sunbin Kim (https://github.com/Einsbon/bipedal-robot-walking-simulation.git) es necesario conocer el entorno de Pybullet. Por lo tanto, veremos cómo es que se importa un modelo CAD y como darle movimiento.
A continuación se muestra el hola mundo de Pybullet, este que código fue obtenido de PyBullet Quickstart Guide
: https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#heading=h.btdfuxtf2f72

In [ ]:
import pybullet as p
import time
import pybullet_data
physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setGravity(0,0,-10)
planeId = p.loadURDF("plane.urdf")
startPos = [0,0,1]
startOrientation = p.getQuaternionFromEuler([0,0,0])
boxId = p.loadURDF("r2d2.urdf",startPos, startOrientation)
#set the center of mass frame (loadURDF sets base link frame) startPos/Ornp.resetBasePositionAndOrientation(boxId, startPos, startOrientation)
for i in range (10000):
    p.stepSimulation()
    time.sleep(1./240.)
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print(cubePos,cubeOrn)
p.disconnect()


# Importando un modelo propio
Para poder importar tu modelo es necesario transformar tu archivo CAD a un formato URDF. En este caso se creó el modelo en SolidWorks y después usando la extensión sw_urdf_exporter. El archivo para importar es un robot manipulador de 2 grados de libertad.

In [ ]:
import pybullet as p
import time
import pybullet_data
import os

path_robot = "C:/Users/osciv/Documents/Practicas/Programa/Learning_Pybullet/R_URDF/ROBOT2GDL.SLDASM/urdf/ROBOT2GDL.SLDASM.urdf"

physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setGravity(0,0,-9.81)
p.resetDebugVisualizerCamera(cameraDistance=2, cameraYaw=90, cameraPitch=0, cameraTargetPosition=[1, 0, 1])
planeId = p.loadURDF("plane.urdf")
startPos = [0,0,1]
startOrientation = p.getQuaternionFromEuler([0,0,0])
robotID = p.loadURDF(path_robot,startPos, startOrientation)
#j = p.getNumJoints(robotID)
p.setJointMotorControl2(robotID,1,p.POSITION_CONTROL,targetPosition=0.0,force=10,maxVelocity=5)
for i in range (10000):
    p.stepSimulation()
    time.sleep(1./240.)
p.disconnect()

# CODIGO PARA DAR MOVIEMIENTO A LAS ARTICULACIONES

Para hacer esto es necesario usar la funcion setJointMotorControl2. Esta funcion recibe como parametros:

| |Name|Type|description|
|---|---|---|---|
|required|bodyUniqueId|int|body unique id as returned from loadURDF etc|
|required|jointIndex|int|link index in range [0..getNumJoints(bodyUniqueId) (note that link index == joint index)|
|required|controlMode|int|POSITION_CONTROL (which is in fact CONTROL_MODE_POSITION_VELOCITY_PD), VELOCITY_CONTROL, TORQUE_CONTROL and PD_CONTROL. (There is also experimental STABLE_PD_CONTROL for stable(implicit) PD control, which requires additional preparation. See humanoidMotionCapture.py and pybullet_envs.deep_mimc for STABLE_PD_CONTROL examples.) TORQUE_CONTROL will apply a torque instantly, so it only is effective when calling stepSimulation explicitly.
|
|optional|targetPosition|float|in POSITION_CONTROL the targetValue is target position of the joint|
|optional|targetVelocity|float|in VELOCITY_CONTROL and POSITION_CONTROL  the targetVelocity is the desired velocity of the joint, see implementation note below. Note that the targetVelocity is not the maximum joint velocity. In PD_CONTROL and POSITION_CONTROL/CONTROL_MODE_POSITION_VELOCITY_PD, the final target velocity is computed using: kp*(erp*(desiredPosition-currentPosition)/dt)+currentVelocity+kd*(m_desiredVelocity - currentVelocity). See also examples/pybullet/examples/pdControl.py|
|required|force|float|in POSITION_CONTROL and VELOCITY_CONTROL this is the maximum motor force used to reach the target value. In TORQUE_CONTROL this is the force/torque to be applied each simulation step.|
|optional|positionGain|float|Gain for position control|
|optional|velocityGain|float|Gain for velocity control|
|optional|maxVelocity|float|in POSITION_CONTROL this limits the velocity to a maximum|
|optional|physicsClientId|int|if you are connected to multiple servers, you can pick one.|

Implenetaremos esta funcion a nuestro modelo para mover las articulaciones de nuestro robot.

In [ ]:
path_robot = "C:/Users/osciv/Documents/Practicas/Programa/Learning_Pybullet/R_URDF/ROBOT2GDL.SLDASM/urdf/ROBOT2GDL.SLDASM.urdf"

physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setGravity(0,0,-9.81)
p.resetDebugVisualizerCamera(cameraDistance=2, cameraYaw=90, cameraPitch=0, cameraTargetPosition=[1, 0, 1])
planeId = p.loadURDF("plane.urdf")
startPos = [0,0,1]
startOrientation = p.getQuaternionFromEuler([0,0,0])
robotID = p.loadURDF(path_robot,[0, 0, 0], useFixedBase=1)

# p.setJointMotorControl2(robotID,joint,controlMode,targetPosition,positionGain,velocityGain,force,maxVelocity,physicsClientId)
p.setJointMotorControl2(
                        robotID,
                        jointIndex = 0,
                        controlMode = p.POSITION_CONTROL,
                        targetVelocity = 45)
for i in range (100000):
    p.stepSimulation()
    time.sleep(1./240.)
p.disconnect()